# Pydantic-based Mesh Example

This notebook demonstrates how to use the new Pydantic-based Mesh class in meshly. It covers:

1. Creating custom Mesh subclasses with additional attributes
2. Working with numpy arrays in Pydantic models
3. Encoding and decoding meshes to/from zip files
4. Optimizing meshes with the built-in optimization methods

In [1]:
import os
import numpy as np
from typing import Optional, List
from pydantic import Field

# Import the Mesh class
from meshly import Mesh, MeshUtils

## 1. Creating a Custom Mesh Subclass

One of the key benefits of the new Pydantic-based Mesh class is the ability to create custom subclasses with additional attributes. Let's create a `TexturedMesh` class that adds texture coordinates and normals.

In [2]:
class TexturedMesh(Mesh):
    """
    A mesh with texture coordinates and normals.
    
    This demonstrates how to create a custom Mesh subclass with additional
    numpy array attributes that will be automatically encoded/decoded.
    """
    # Add texture coordinates and normals as additional numpy arrays
    texture_coords: np.ndarray = Field(..., description="Texture coordinates")
    normals: Optional[np.ndarray] = Field(None, description="Vertex normals")
    
    # Add non-array attributes
    material_name: str = Field("default", description="Material name")
    tags: List[str] = Field(default_factory=list, description="Tags for the mesh")

## 2. Creating a Mesh Instance

Now let's create a simple cube mesh with texture coordinates and normals.

In [3]:
# Create vertices for a cube
vertices = np.array([
    [-0.5, -0.5, -0.5],  # 0: bottom-left-back
    [0.5, -0.5, -0.5],   # 1: bottom-right-back
    [0.5, 0.5, -0.5],    # 2: top-right-back
    [-0.5, 0.5, -0.5],   # 3: top-left-back
    [-0.5, -0.5, 0.5],   # 4: bottom-left-front
    [0.5, -0.5, 0.5],    # 5: bottom-right-front
    [0.5, 0.5, 0.5],     # 6: top-right-front
    [-0.5, 0.5, 0.5]     # 7: top-left-front
], dtype=np.float32)

# Create indices for the cube
indices = np.array([
    0, 1, 2, 2, 3, 0,  # back face
    1, 5, 6, 6, 2, 1,  # right face
    5, 4, 7, 7, 6, 5,  # front face
    4, 0, 3, 3, 7, 4,  # left face
    3, 2, 6, 6, 7, 3,  # top face
    4, 5, 1, 1, 0, 4   # bottom face
], dtype=np.uint32)

# Create texture coordinates (one for each vertex)
texture_coords = np.array([
    [0.0, 0.0],  # 0
    [1.0, 0.0],  # 1
    [1.0, 1.0],  # 2
    [0.0, 1.0],  # 3
    [0.0, 0.0],  # 4
    [1.0, 0.0],  # 5
    [1.0, 1.0],  # 6
    [0.0, 1.0]   # 7
], dtype=np.float32)

# Create normals (one for each vertex)
normals = np.array([
    [0.0, 0.0, -1.0],  # 0: back
    [0.0, 0.0, -1.0],  # 1: back
    [0.0, 0.0, -1.0],  # 2: back
    [0.0, 0.0, -1.0],  # 3: back
    [0.0, 0.0, 1.0],   # 4: front
    [0.0, 0.0, 1.0],   # 5: front
    [0.0, 0.0, 1.0],   # 6: front
    [0.0, 0.0, 1.0]    # 7: front
], dtype=np.float32)

# Create the textured mesh
mesh = TexturedMesh(
    vertices=vertices,
    indices=indices,
    texture_coords=texture_coords,
    normals=normals,
    material_name="cube_material",
    tags=["cube", "example"]
)

print(f"Mesh created with {mesh.vertex_count} vertices and {mesh.index_count} indices")
print(f"Material name: {mesh.material_name}")
print(f"Tags: {mesh.tags}")

Mesh created with 8 vertices and 36 indices
Material name: cube_material
Tags: ['cube', 'example']


## 3. Optimizing the Mesh

The Mesh class provides several optimization methods that can be used to improve rendering performance.

In [4]:
# Optimize the mesh for vertex cache
MeshUtils.optimize_vertex_cache(mesh)
print("Optimized for vertex cache")

# Optimize the mesh for overdraw
MeshUtils.optimize_overdraw(mesh)
print("Optimized for overdraw")

# Optimize the mesh for vertex fetch
MeshUtils.optimize_vertex_fetch(mesh)
print("Optimized for vertex fetch")

Optimized for vertex cache
Optimized for overdraw
Optimized for vertex fetch


## 4. Encoding and Saving the Mesh

The Mesh class provides methods for encoding the mesh and saving it to a zip file.

In [5]:
# Encode the mesh
encoded_data = MeshUtils.encode(mesh)
print(f"Encoded mesh: {len(encoded_data['mesh'].vertices)} bytes for vertices, {len(encoded_data['mesh'].indices)} bytes for indices")
print(f"Encoded arrays: {list(encoded_data['arrays'].keys())}")

# Save the mesh to a zip file
zip_path = "textured_cube.zip"
MeshUtils.save_to_zip(mesh, zip_path)
print(f"Saved mesh to {zip_path}, file size: {os.path.getsize(zip_path)} bytes")

Encoded mesh: 67 bytes for vertices, 29 bytes for indices
Encoded arrays: ['texture_coords', 'normals']
Saved mesh to textured_cube.zip, file size: 1248 bytes


## 5. Loading the Mesh from a Zip File

The Mesh class provides a class method for loading a mesh from a zip file.

In [6]:
# Load the mesh from the zip file
loaded_mesh = MeshUtils.load_from_zip(TexturedMesh, zip_path)
print(f"Loaded mesh: {loaded_mesh.vertex_count} vertices, {loaded_mesh.index_count} indices")
print(f"Material name: {loaded_mesh.material_name}")
print(f"Tags: {loaded_mesh.tags}")

# Verify that the texture coordinates and normals were loaded correctly
print(f"\nTexture coordinates shape: {loaded_mesh.texture_coords.shape}")
print(f"Normals shape: {loaded_mesh.normals.shape}")

Loaded mesh: 8 vertices, 36 indices
Material name: cube_material
Tags: ['cube', 'example']

Texture coordinates shape: (8, 2)
Normals shape: (8, 3)


## 6. Creating a Different Mesh Subclass

Let's create another mesh subclass with different attributes to demonstrate the flexibility of the Pydantic-based Mesh class.

In [7]:
class SkinnedMesh(Mesh):
    """
    A mesh with skinning information for animation.
    """
    # Add bone weights and indices as additional numpy arrays
    bone_weights: np.ndarray = Field(..., description="Bone weights for each vertex")
    bone_indices: np.ndarray = Field(..., description="Bone indices for each vertex")
    
    # Add non-array attributes
    skeleton_name: str = Field("default", description="Skeleton name")
    animation_names: List[str] = Field(default_factory=list, description="Animation names")

# Create a simple skinned mesh
skinned_mesh = SkinnedMesh(
    vertices=vertices,
    indices=indices,
    bone_weights=np.random.random((len(vertices), 4)).astype(np.float32),  # 4 weights per vertex
    bone_indices=np.random.randint(0, 4, (len(vertices), 4)).astype(np.uint8),  # 4 bone indices per vertex
    skeleton_name="human_skeleton",
    animation_names=["walk", "run", "jump"]
)

print(f"Skinned mesh created with {skinned_mesh.vertex_count} vertices and {skinned_mesh.index_count} indices")
print(f"Skeleton name: {skinned_mesh.skeleton_name}")
print(f"Animation names: {skinned_mesh.animation_names}")
print(f"Bone weights shape: {skinned_mesh.bone_weights.shape}")
print(f"Bone indices shape: {skinned_mesh.bone_indices.shape}")

Skinned mesh created with 8 vertices and 36 indices
Skeleton name: human_skeleton
Animation names: ['walk', 'run', 'jump']
Bone weights shape: (8, 4)
Bone indices shape: (8, 4)


## 7. Saving and Loading the Skinned Mesh

Let's save and load the skinned mesh to demonstrate that all attributes are preserved.

In [10]:
# Save the skinned mesh to a zip file
skinned_zip_path = "skinned_cube.zip"
MeshUtils.save_to_zip(skinned_mesh, skinned_zip_path)
print(f"Saved skinned mesh to {skinned_zip_path}, file size: {os.path.getsize(skinned_zip_path)} bytes")

# Load the skinned mesh from the zip file
loaded_skinned_mesh = MeshUtils.load_from_zip(SkinnedMesh, skinned_zip_path)
print(f"\nLoaded skinned mesh: {loaded_skinned_mesh.vertex_count} vertices, {loaded_skinned_mesh.index_count} indices")
print(f"Skeleton name: {loaded_skinned_mesh.skeleton_name}")
print(f"Animation names: {loaded_skinned_mesh.animation_names}")
print(f"Bone weights shape: {loaded_skinned_mesh.bone_weights.shape}")
print(f"Bone indices shape: {loaded_skinned_mesh.bone_indices.shape}")

Saved skinned mesh to skinned_cube.zip, file size: 1401 bytes

Loaded skinned mesh: 8 vertices, 36 indices
Skeleton name: human_skeleton
Animation names: ['walk', 'run', 'jump']
Bone weights shape: (8, 4)
Bone indices shape: (8, 4)


## 8. Cleaning Up

Let's clean up the files we created.

In [9]:
# Clean up
for path in [zip_path, skinned_zip_path]:
    if os.path.exists(path):
        os.remove(path)
        print(f"Removed {path}")

print("\nExample completed successfully!")

Removed textured_cube.zip
Removed skinned_cube.zip

Example completed successfully!
